<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/binary_classification_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델의 depth와 활성함수의 중첩, 그에 따른 비선형성 증가의 의미.

딥러닝 모델의 깊어질수록 활성함수가 중첩되고 모델전체의 비선형성이 증가하게 된다. 비선형성이 증가함에따라 모델이 그리는 곡면은 더 복잡해지게 되므로 SGD같이 local minimum을 탈출할 수단이 없는 optimizer는 local minimum에 빠질 확률이 커지게 될 것 이다. 이번 프로젝트에서는 이러한 현상을 시각화해 깊이 이해하고 이를 극복하기위한 여러 optimize를 활용해 보고자 한다.

# Data 생성

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
# case 1 키, 몸무게
N=20
random0=torch.randn(int(N/2),1)
random5=torch.randn(int(N/2),1)+8

class1_data=torch.hstack([random0,random5])
class2_data=torch.hstack([random5,random0])
# class3_data=torch.hstack([random5,random0+8])

class1_label=torch.ones(int(N/2),1)
class2_label=torch.zeros(int(N/2),1)
class3_label=torch.ones(int(N/2),1)

X=torch.vstack([class1_data,class2_data])
y=torch.vstack([class1_label,class2_label])

# X=torch.vstack([class1_data,class2_data,class3_data])
# y=torch.vstack([class1_label,class2_label,class3_label])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(class1_data[:,0],class1_data[:,1],'bo')
plt.plot(class2_data[:,0],class2_data[:,1],'ro')
# plt.plot(class3_data[:,0],class3_data[:,1],'bo')
plt.xlabel('x1')
plt.ylabel('x2')
plt.grid()

# 모델생성과 학습

비슷한 수의 parameter(각각 300개, 330개)을 갖지만 깊이가 다른(각각 2층, 5층) model 2개를 생성한다.

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Sequential(nn.Linear(2, 100),
                                    nn.Sigmoid(),
                                    nn.Linear(100, 1),
                                    nn.Sigmoid()
                                    )

        self.linear2 = nn.Sequential(nn.Linear(2, 10),
                                    nn.Sigmoid(),
                                    nn.Linear(10, 10),
                                    nn.Sigmoid(),
                                    nn.Linear(10, 10),
                                    nn.Sigmoid(),
                                    nn.Linear(10, 10),
                                    nn.Sigmoid(),
                                    nn.Linear(10, 1),
                                    nn.Sigmoid()
                                    )

    def forward(self, x):
        x1 = self.linear1(x)
        x2 = self.linear2(x)

        return x1, x2

두 모델을 SGD로 학습시키고 진행에 따른 Loss값 변화를 그래프로 비교한다.

In [ ]:
model = Model()

LR = 1e-1
EPOC = 100

# optimizer = optim.Adam(model.parameters(), lr = LR)
optimizer = optim.SGD(model.parameters(), lr = LR)

loss1_history = []
loss2_history = [] # 추가

model.train()
for ep in range(EPOC):
    y1_hat, y2_hat = model(X)

    loss1 = F.binary_cross_entropy(y1_hat, y)
    loss2 = F.binary_cross_entropy(y2_hat, y) # 추가

    optimizer.zero_grad()
    loss1.backward()
    loss2.backward() # 추가
    optimizer.step()

    loss1_history += [loss1.item()]
    loss2_history += [loss2.item()] # 추가

plt.plot(range(1,EPOC+1), loss1_history)
plt.plot(range(1,EPOC+1), loss2_history)
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
# x1_test=torch.linspace(-10,10,30) # case 1
# x2_test=torch.linspace(-10,10,30) # case 1
x1_test = torch.linspace(-10,10,30)
x2_test = torch.linspace(-10,10,30)

X1_test, X2_test = torch.meshgrid(x1_test, x2_test)
# print(X1_test.shape)

X_test = torch.cat([X1_test.unsqueeze(dim=2), X2_test.unsqueeze(dim=2)], dim=2)
# print(X_test)

model.eval()

with torch.no_grad():
    y_hat = model(X_test)

Y_hat = y_hat.squeeze()


plt.figure(figsize=[10, 9]) # figsize=[가로, 세로]
ax = plt.axes(projection="3d")
ax.view_init(elev=25,azim=-140)
ax.plot_surface(X1_test,X2_test, Y_hat.numpy(), cmap="viridis", alpha=0.2)
plt.plot(class1_data[:,0],class1_data[:,1],class1_label.squeeze(),'bo')
plt.plot(class2_data[:,0],class2_data[:,1],class2_label.squeeze(),'ro')
# plt.plot(class3_data[:,0],class3_data[:,1],class3_label.squeeze(),'bo')
plt.xlabel("x1")
plt.ylabel("x2")